# <center>Generating Music Using GAN</center>

This workbook will implement modified code from [this](https://github.com/corynguyen19/midi-lstm-gan) GitHub repo.


In [1]:
from __future__ import print_function, division
import sys
import matplotlib.pyplot as plt
import numpy as np
import pickle
import glob
import time
import os
from music21 import converter, instrument, note, chord, stream
from keras.layers import Input, Dense, Reshape, Dropout, CuDNNLSTM, Bidirectional
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import np_utils

Using TensorFlow backend.


## Loading and Cleaning the Data

First, I will load all the notes from the midi files

In [117]:
def get_notes(path):
    """
        Gets all notes and chords from midi files
    """
    notes = []

    for file in glob.glob(path + "*.mid"):        
        song = []
        midi = converter.parse(file)
        
        print("Parsing %s" % file)
        
        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                song.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                song.append('.'.join(str(n) for n in element.normalOrder))
        notes.append(song)

    return notes

def get_notes_with_key(path, filter_key, mode):
    """
        Gets all notes and chords from midi files where the key matches the string input

        Parameters
        ----------
        path : str
            The path to the file
        filter_key : str
            The string to filter the key on
        mode : int
            The type of key used where:
                0 - key
                1 - major/minor
                else - key and major/minor
    """
    notes = []

    for file in glob.glob(path + "*.mid"):        
        song = []
        midi = converter.parse(file)
        
        # Only use music of the same key
        key = midi.analyze('key')
        if(mode==0):
            key_string = str(key.tonic.name)
        elif(mode==1):
            key_string = str(key.mode)
        else:
            key_string = str(key.tonic.name + key.mode)
            
        if(key_string==filter_key):
            print("Parsing %s" % file)
            notes_to_parse = None

            try: # file has instrument parts
                s2 = instrument.partitionByInstrument(midi)
                notes_to_parse = s2.parts[0].recurse() 
            except: # file has notes in a flat structure
                notes_to_parse = midi.flat.notes

            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    song.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    song.append('.'.join(str(n) for n in element.normalOrder))
            notes.append(song)

    return notes

""" Train a Neural Network to generate music """
# Get notes from midi files
input_dir_choice = 0
input_dir_names = ["Pokemon", "LoZ", "Pokemon GSC", "ABBA"]

input_path = "../" + input_dir_names[input_dir_choice] + " MIDIs/"
# example of each mode: 0 - C, 1 - major, 2 - Cmajor
notes = get_notes_with_key(input_path, "minor", 1)
# notes = get_notes(input_path)

Parsing ../Pokemon MIDIs\Pokemon - Farewell, Pikachu!.mid
Parsing ../Pokemon MIDIs\Pokemon - Lugias Song.mid
Parsing ../Pokemon MIDIs\Pokemon - Oracion.mid
Parsing ../Pokemon MIDIs\Pokemon - Pallet Town.mid
Parsing ../Pokemon MIDIs\Pokemon - The Ghost at Maiden's Peak.mid
Parsing ../Pokemon MIDIs\Pokemon Black 2White 2 - Ns Room.mid
Parsing ../Pokemon MIDIs\Pokemon BlackWhite - Battle CherenBianca.mid
Parsing ../Pokemon MIDIs\Pokemon BlackWhite - Battle Elite Four.mid
Parsing ../Pokemon MIDIs\Pokemon BlackWhite - Battle Team Plasma.mid
Parsing ../Pokemon MIDIs\Pokemon BlackWhite - Champion Alder.mid
Parsing ../Pokemon MIDIs\Pokemon BlackWhite - Driftveil City.mid
Parsing ../Pokemon MIDIs\Pokemon BlackWhite - Ending Onward to Our Own Futures.mid
Parsing ../Pokemon MIDIs\Pokemon BlackWhite - Mistralton City.mid
Parsing ../Pokemon MIDIs\Pokemon BlackWhite - N the Pokemon Child.mid
Parsing ../Pokemon MIDIs\Pokemon BlackWhite - Ns Castle.mid
Parsing ../Pokemon MIDIs\Pokemon BlackWhite - Pok

I will now use an algo to determine the key of each song

In [3]:
def print_key(path):
    key_count = dict()
    for file in glob.glob(path + "*.mid"):
        print("Parsing %s" % file)
        
        song = []
        midi = converter.parse(file)
        
        key = midi.analyze('key')
        key_string = key.tonic.name + key.mode
        if (key_string in key_count): 
            key_count[key_string] += 1
        else: 
            key_count[key_string] = 1
        print(key.tonic.name, key.mode)
    return key_count

input_dir_choice = 1
input_dir_names = ["Pokemon", "LoZ", "Pokemon GSC", "ABBA"]

input_path = "../" + input_dir_names[input_dir_choice] + " MIDIs/"
key_count = print_key(input_path)
key_count

Parsing ../Pokemon GSC MIDIs\Pokemon Gold, Silver, Crystal - Cinnabar Island (HGSS Version).mid
G major
Parsing ../Pokemon GSC MIDIs\Pokemon Gold, Silver, Crystal - S.S. Aqua .mid
G major
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Azalea TownBlackthorn City.mid
C# major
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Bicycle.mid
E minor
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Bug Catching Contest.mid
E minor
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Burned Tower.mid
E minor
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Champion Battle.mid
G# minor
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Cherrygrove CityMahogany Town.mid
F major
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Dance Theatre.mid
A minor
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Dark Cave.mid
A- major
Parsing ../Pokemon GSC MIDIs\Pokemon GoldSilverCrystal - Dragons Den.mid
C# minor
Parsing ../Pokemon GSC MIDIs\Pokemon

{'Gmajor': 5,
 'C#major': 1,
 'Eminor': 3,
 'G#minor': 3,
 'Fmajor': 3,
 'Aminor': 3,
 'A-major': 4,
 'C#minor': 3,
 'Cmajor': 8,
 'Emajor': 4,
 'Dmajor': 8,
 'B-minor': 4,
 'E-major': 3,
 'Bmajor': 2,
 'Amajor': 2,
 'Fminor': 1,
 'Cminor': 1,
 'Dminor': 1,
 'F#major': 1,
 'E-minor': 1}

Next, I will find all possible notes and use this to determine how to alter the data to a machine readable format.

In [118]:
possibleNotes = set([item for sublist in notes for item in sublist])

n_vocab = len(possibleNotes)
possibleNotes, n_vocab

({'0',
  '0.1',
  '0.1.3',
  '0.1.5',
  '0.1.6',
  '0.2',
  '0.2.4.7.9',
  '0.2.5',
  '0.2.5.7',
  '0.2.6',
  '0.2.7',
  '0.3',
  '0.3.4',
  '0.3.5',
  '0.3.5.8',
  '0.3.6',
  '0.3.6.7',
  '0.3.7',
  '0.4',
  '0.4.5',
  '0.4.5.7',
  '0.4.7',
  '0.4.8',
  '0.5',
  '0.6',
  '1',
  '1.2',
  '1.2.4',
  '1.2.5',
  '1.2.5.8',
  '1.2.6',
  '1.2.7',
  '1.3',
  '1.3.4',
  '1.3.4.7',
  '1.3.5',
  '1.3.6',
  '1.3.8',
  '1.4',
  '1.4.5',
  '1.4.6',
  '1.4.6.9',
  '1.4.7',
  '1.4.7.9.10',
  '1.4.8',
  '1.5',
  '1.5.6',
  '1.5.7',
  '1.5.8',
  '1.5.9',
  '1.6',
  '1.6.7',
  '1.7',
  '10',
  '10.0',
  '10.0.1',
  '10.0.2',
  '10.0.2.4.7',
  '10.0.2.5',
  '10.0.3',
  '10.0.3.5',
  '10.0.4',
  '10.0.5',
  '10.1',
  '10.1.2.5',
  '10.1.3',
  '10.1.3.6',
  '10.1.4',
  '10.1.4.5.6',
  '10.1.4.6',
  '10.1.5',
  '10.11',
  '10.11.1',
  '10.11.2',
  '10.11.2.6',
  '10.11.3',
  '10.2',
  '10.2.3',
  '10.2.4',
  '10.2.5',
  '10.3',
  '11',
  '11.0',
  '11.0.2',
  '11.0.4',
  '11.0.4.7',
  '11.1',
  '11.1.2',
 

Now I will prepare the sequences of notes by looking at each song individually. I will first grab an arrays of size **sequence_length** with a stride of **step_size** from each song. Then I will map the chords to integers so the model can learn from that and normalize the input between 0-1.

In [119]:
def prepare_sequences(notes, allNotes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100
    step_size = 1

    # get all pitch names
    pitchnames = sorted(allNotes)

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for song in notes:
        for i in range(0, len(song) - sequence_length, step_size):
            sequence_in = song[i:i + sequence_length]
            sequence_out = song[i + sequence_length]
            network_input.append([note_to_int[char] for char in sequence_in])
            network_output.append(note_to_int[sequence_out])

    # reshape the input into a format compatible with LSTM layers
    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    
    # normalize input between 0 and 1
    network_input = network_input / float(n_vocab)
    
    # normalize input between -1 and 1
#     network_input = (network_input / float(n_vocab))*2 - 1

    # Make one-hot-encoding
    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

network_input, network_output = prepare_sequences(notes, possibleNotes, n_vocab)
len(network_input)

55548

## Constructing the model


In [120]:
class GAN():
    def __init__(self, rows, network_input):
        self.seq_length = rows
        self.seq_shape = (self.seq_length, 1)
        self.latent_dim = 1000
        self.disc_loss = []
        self.gen_loss =[]
        
        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator(network_input)
        self.discriminator.trainable = True
        self.discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates note sequences
        z = Input(shape=(self.latent_dim,))
        generated_seq = self.generator(z)

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(generated_seq)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        
        self.outputDest = '../output/GAN_' + str(int(time.time())) + '/'
        if not os.path.exists(self.outputDest):
            os.makedirs(self.outputDest)

    def build_discriminator(self, network_input):

        model = Sequential()
        model.add(CuDNNLSTM(512, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
        model.add(Dropout(0.3))
        model.add(Bidirectional(CuDNNLSTM(512, return_sequences=True)))
        model.add(Dropout(0.3))
        model.add(Bidirectional(CuDNNLSTM(512)))
        model.add(Dense(256))
        model.add(Dropout(0.3))
        model.add(Dense(1, activation='sigmoid'))
        
        model.summary()

        seq = Input(shape=self.seq_shape)
        validity = model(seq)

        return Model(seq, validity)
      
    def build_generator(self):

        model = Sequential()
        model.add(Dense(128, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.seq_shape), activation='tanh'))
        model.add(Reshape(self.seq_shape))
        model.summary()
        
        noise = Input(shape=(self.latent_dim,))
        seq = model(noise)

        return Model(noise, seq)

    def train(self, X_train, y_train, epochs, batch_size=128, sample_interval=50):

        # Adversarial ground truths
        real = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        # Training the model
        for epoch in range(epochs+1):

            # Training the discriminator
            # Select a random batch of note sequences
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            real_seqs = X_train[idx]

            #noise = np.random.choice(range(484), (batch_size, self.latent_dim))
            #noise = (noise-242)/242
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new note sequences
            gen_seqs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(real_seqs, real)
            d_loss_fake = self.discriminator.train_on_batch(gen_seqs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            #  Training the Generator
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as real)
            g_loss = self.combined.train_on_batch(noise, real)

            # Print the progress and save into loss lists
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            self.disc_loss.append(d_loss[0])
            self.gen_loss.append(g_loss)
            
              # Save model often
            if epoch % sample_interval == 0:
                self.generator.save(self.outputDest + "GANmodel_weights_" + str(epoch) + ".hdf5")
                
    def generate_notes(self, input_notes):
        # Get pitch names and store in a dictionary
        notes = input_notes
        pitchnames = sorted(notes)
        n_vocab = len(pitchnames)
        int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
        
        # Use random noise to generate sequences
        noise = np.random.normal(0, 1, (1, self.latent_dim))
        predictions = self.generator.predict(noise)
        
        print(predictions)
        pred_notes = [(x+1)*n_vocab/2 for x in predictions[0]]
        pred_notes = [int_to_note[min(int(x), n_vocab - 1)] for x in pred_notes]
        print(pred_notes)
        
        return pred_notes
        
    def create_midi(self, prediction_output, filename):
        offset = 0
        output_notes = []

        # create note and chord objects based on the values generated by the model
        for pattern in prediction_output:
            # pattern is a chord
            if ('.' in pattern) or pattern.isdigit():
                notes_in_chord = pattern.split('.')
                notes = []
                for current_note in notes_in_chord:
                    new_note = note.Note(int(current_note))
                    new_note.storedInstrument = instrument.Piano()
                    notes.append(new_note)
                new_chord = chord.Chord(notes)
                new_chord.offset = offset
                output_notes.append(new_chord)
            # pattern is a note
            else:
                new_note = note.Note(pattern)
                new_note.offset = offset
                new_note.storedInstrument = instrument.Piano()
                output_notes.append(new_note)
            # increase offset each iteration so that notes do not stack
            offset += 0.5

        midi_stream = stream.Stream(output_notes)
        midi_stream.write('midi', fp='{}.mid'.format(filename))
        
    def generate_midi_from_saves(self, input_notes):
        # Have each model make a song
        count = 0
        for model_path in glob.glob(self.outputDest + "*.hdf5"):
            print("Composing from %s" % model_path)
            self.generator.load_weights(model_path)
            prediction_notes = self.generate_notes(input_notes)
            self.create_midi(prediction_notes, self.outputDest + 'GAN_output_' + str(count))
            print(self.outputDest + 'GAN_output_' + str(count))
            count += 1
        
    def plot_loss(self):
        plt.plot(self.disc_loss, c='red')
        plt.plot(self.gen_loss, c='blue')
        plt.title("GAN Loss per Epoch")
        plt.legend(['Discriminator', 'Generator'])
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.savefig(self.outputDest + 'GAN_Loss_per_Epoch_final.png', transparent=True)
        plt.close()

In [121]:
gan = GAN(100, network_input)
gan.train(X_train=network_input, y_train=network_output, epochs=2000, batch_size=20, sample_interval=100)
gan.plot_loss()

Model: "sequential_85"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_126 (CuDNNLSTM)   (None, 100, 512)          1054720   
_________________________________________________________________
dropout_36 (Dropout)         (None, 100, 512)          0         
_________________________________________________________________
bidirectional_48 (Bidirectio (None, 100, 1024)         4202496   
_________________________________________________________________
dropout_37 (Dropout)         (None, 100, 1024)         0         
_________________________________________________________________
bidirectional_49 (Bidirectio (None, 1024)              6299648   
_________________________________________________________________
dense_221 (Dense)            (None, 256)               262400    
_________________________________________________________________
dropout_38 (Dropout)         (None, 256)             

98 [D loss: 1.140641, acc.: 50.00%] [G loss: 0.187356]
99 [D loss: 1.090166, acc.: 50.00%] [G loss: 0.180620]
100 [D loss: 1.065862, acc.: 50.00%] [G loss: 0.195759]
101 [D loss: 1.248775, acc.: 50.00%] [G loss: 0.192858]
102 [D loss: 1.129633, acc.: 50.00%] [G loss: 0.185198]
103 [D loss: 1.176300, acc.: 50.00%] [G loss: 0.173351]
104 [D loss: 1.195978, acc.: 50.00%] [G loss: 0.179998]
105 [D loss: 1.173565, acc.: 50.00%] [G loss: 0.181206]
106 [D loss: 1.141919, acc.: 50.00%] [G loss: 0.182831]
107 [D loss: 1.136107, acc.: 50.00%] [G loss: 0.165078]
108 [D loss: 1.056787, acc.: 50.00%] [G loss: 0.177542]
109 [D loss: 1.217332, acc.: 50.00%] [G loss: 0.176364]
110 [D loss: 1.078933, acc.: 50.00%] [G loss: 0.178169]
111 [D loss: 1.079522, acc.: 50.00%] [G loss: 0.179682]
112 [D loss: 1.156452, acc.: 50.00%] [G loss: 0.163160]
113 [D loss: 1.104455, acc.: 50.00%] [G loss: 0.169872]
114 [D loss: 1.139707, acc.: 50.00%] [G loss: 0.194294]
115 [D loss: 1.103998, acc.: 50.00%] [G loss: 0.16

245 [D loss: 1.099331, acc.: 50.00%] [G loss: 0.213076]
246 [D loss: 1.098618, acc.: 50.00%] [G loss: 0.227182]
247 [D loss: 1.155793, acc.: 50.00%] [G loss: 0.212120]
248 [D loss: 1.115106, acc.: 50.00%] [G loss: 0.201664]
249 [D loss: 1.144189, acc.: 50.00%] [G loss: 0.224885]
250 [D loss: 1.159521, acc.: 50.00%] [G loss: 0.234969]
251 [D loss: 1.178276, acc.: 50.00%] [G loss: 0.193360]
252 [D loss: 1.068588, acc.: 50.00%] [G loss: 0.190795]
253 [D loss: 1.188911, acc.: 50.00%] [G loss: 0.240701]
254 [D loss: 1.108306, acc.: 50.00%] [G loss: 0.206176]
255 [D loss: 1.156240, acc.: 50.00%] [G loss: 0.207085]
256 [D loss: 1.088064, acc.: 50.00%] [G loss: 0.219357]
257 [D loss: 1.115626, acc.: 50.00%] [G loss: 0.212411]
258 [D loss: 1.122637, acc.: 50.00%] [G loss: 0.198370]
259 [D loss: 1.159214, acc.: 50.00%] [G loss: 0.216815]
260 [D loss: 1.157787, acc.: 50.00%] [G loss: 0.233653]
261 [D loss: 1.207959, acc.: 50.00%] [G loss: 0.201800]
262 [D loss: 1.106004, acc.: 50.00%] [G loss: 0.

392 [D loss: 1.146105, acc.: 50.00%] [G loss: 0.220098]
393 [D loss: 1.116915, acc.: 50.00%] [G loss: 0.224038]
394 [D loss: 1.154768, acc.: 50.00%] [G loss: 0.202413]
395 [D loss: 1.089069, acc.: 50.00%] [G loss: 0.176621]
396 [D loss: 1.100200, acc.: 50.00%] [G loss: 0.167243]
397 [D loss: 1.169474, acc.: 50.00%] [G loss: 0.208296]
398 [D loss: 1.162279, acc.: 50.00%] [G loss: 0.194979]
399 [D loss: 1.098979, acc.: 50.00%] [G loss: 0.247730]
400 [D loss: 1.097056, acc.: 50.00%] [G loss: 0.226016]
401 [D loss: 1.148838, acc.: 50.00%] [G loss: 0.172803]
402 [D loss: 1.187852, acc.: 50.00%] [G loss: 0.167059]
403 [D loss: 1.143482, acc.: 50.00%] [G loss: 0.187294]
404 [D loss: 1.056277, acc.: 50.00%] [G loss: 0.226428]
405 [D loss: 1.102779, acc.: 50.00%] [G loss: 0.215375]
406 [D loss: 1.214560, acc.: 50.00%] [G loss: 0.179201]
407 [D loss: 1.122882, acc.: 50.00%] [G loss: 0.223627]
408 [D loss: 1.162974, acc.: 50.00%] [G loss: 0.199000]
409 [D loss: 1.153663, acc.: 50.00%] [G loss: 0.

539 [D loss: 1.153432, acc.: 50.00%] [G loss: 0.204746]
540 [D loss: 1.112991, acc.: 50.00%] [G loss: 0.195286]
541 [D loss: 1.148707, acc.: 50.00%] [G loss: 0.214713]
542 [D loss: 1.167047, acc.: 50.00%] [G loss: 0.193033]
543 [D loss: 1.088917, acc.: 50.00%] [G loss: 0.188430]
544 [D loss: 1.160033, acc.: 50.00%] [G loss: 0.197177]
545 [D loss: 1.108674, acc.: 50.00%] [G loss: 0.188814]
546 [D loss: 1.108483, acc.: 50.00%] [G loss: 0.187699]
547 [D loss: 1.116268, acc.: 50.00%] [G loss: 0.202383]
548 [D loss: 1.139483, acc.: 50.00%] [G loss: 0.194662]
549 [D loss: 1.138335, acc.: 50.00%] [G loss: 0.181115]
550 [D loss: 1.161405, acc.: 50.00%] [G loss: 0.187356]
551 [D loss: 1.080042, acc.: 50.00%] [G loss: 0.212918]
552 [D loss: 1.153148, acc.: 50.00%] [G loss: 0.197392]
553 [D loss: 1.120827, acc.: 50.00%] [G loss: 0.198387]
554 [D loss: 1.110265, acc.: 50.00%] [G loss: 0.198068]
555 [D loss: 1.093228, acc.: 50.00%] [G loss: 0.182008]
556 [D loss: 1.077200, acc.: 50.00%] [G loss: 0.

...

1413 [D loss: 1.060641, acc.: 50.00%] [G loss: 0.179390]
1414 [D loss: 1.099853, acc.: 50.00%] [G loss: 0.170778]
1415 [D loss: 1.049056, acc.: 50.00%] [G loss: 0.201659]
1416 [D loss: 1.048932, acc.: 50.00%] [G loss: 0.213262]
1417 [D loss: 1.060270, acc.: 50.00%] [G loss: 0.206307]
1418 [D loss: 1.068589, acc.: 50.00%] [G loss: 0.189308]
1419 [D loss: 1.066114, acc.: 50.00%] [G loss: 0.199167]
1420 [D loss: 1.083888, acc.: 50.00%] [G loss: 0.206315]
1421 [D loss: 1.071486, acc.: 50.00%] [G loss: 0.200802]
1422 [D loss: 1.131470, acc.: 50.00%] [G loss: 0.214398]
1423 [D loss: 1.088288, acc.: 50.00%] [G loss: 0.204278]
1424 [D loss: 1.108850, acc.: 50.00%] [G loss: 0.211503]
1425 [D loss: 1.088207, acc.: 50.00%] [G loss: 0.201259]
1426 [D loss: 1.036063, acc.: 50.00%] [G loss: 0.183251]
1427 [D loss: 1.086815, acc.: 50.00%] [G loss: 0.210138]
1428 [D loss: 1.068403, acc.: 50.00%] [G loss: 0.199959]
1429 [D loss: 1.078300, acc.: 50.00%] [G loss: 0.202983]
1430 [D loss: 1.073143, acc.: 5

1557 [D loss: 1.015538, acc.: 50.00%] [G loss: 0.405692]
1558 [D loss: 1.122658, acc.: 50.00%] [G loss: 0.497167]
1559 [D loss: 1.043429, acc.: 50.00%] [G loss: 0.532201]
1560 [D loss: 1.016835, acc.: 50.00%] [G loss: 0.479841]
1561 [D loss: 0.970688, acc.: 50.00%] [G loss: 0.493041]
1562 [D loss: 1.034508, acc.: 50.00%] [G loss: 0.451696]
1563 [D loss: 1.048154, acc.: 50.00%] [G loss: 0.445061]
1564 [D loss: 0.991998, acc.: 50.00%] [G loss: 0.471191]
1565 [D loss: 0.987519, acc.: 50.00%] [G loss: 0.583528]
1566 [D loss: 1.100087, acc.: 50.00%] [G loss: 0.434397]
1567 [D loss: 1.070229, acc.: 50.00%] [G loss: 0.400826]
1568 [D loss: 1.029262, acc.: 50.00%] [G loss: 0.451428]
1569 [D loss: 0.970862, acc.: 50.00%] [G loss: 0.493321]
1570 [D loss: 1.000936, acc.: 50.00%] [G loss: 0.466856]
1571 [D loss: 1.002771, acc.: 50.00%] [G loss: 0.499996]
1572 [D loss: 1.097313, acc.: 50.00%] [G loss: 0.522300]
1573 [D loss: 1.088694, acc.: 50.00%] [G loss: 0.523260]
1574 [D loss: 1.104833, acc.: 5

1701 [D loss: 1.084211, acc.: 50.00%] [G loss: 0.407199]
1702 [D loss: 1.021919, acc.: 50.00%] [G loss: 0.378627]
1703 [D loss: 1.075430, acc.: 50.00%] [G loss: 0.325434]
1704 [D loss: 1.038058, acc.: 50.00%] [G loss: 0.395775]
1705 [D loss: 0.981167, acc.: 50.00%] [G loss: 0.438123]
1706 [D loss: 1.125288, acc.: 50.00%] [G loss: 0.373137]
1707 [D loss: 1.060760, acc.: 50.00%] [G loss: 0.364919]
1708 [D loss: 1.031585, acc.: 50.00%] [G loss: 0.336340]
1709 [D loss: 1.110040, acc.: 50.00%] [G loss: 0.353430]
1710 [D loss: 1.046861, acc.: 50.00%] [G loss: 0.415110]
1711 [D loss: 0.974061, acc.: 50.00%] [G loss: 0.434352]
1712 [D loss: 1.086082, acc.: 50.00%] [G loss: 0.308839]
1713 [D loss: 0.976233, acc.: 50.00%] [G loss: 0.370654]
1714 [D loss: 1.043069, acc.: 50.00%] [G loss: 0.398642]
1715 [D loss: 1.063722, acc.: 50.00%] [G loss: 0.401342]
1716 [D loss: 0.986963, acc.: 50.00%] [G loss: 0.355781]
1717 [D loss: 1.051336, acc.: 50.00%] [G loss: 0.405545]
1718 [D loss: 1.076569, acc.: 5

1845 [D loss: 0.983563, acc.: 50.00%] [G loss: 0.390055]
1846 [D loss: 1.142299, acc.: 50.00%] [G loss: 0.341473]
1847 [D loss: 1.035327, acc.: 50.00%] [G loss: 0.335411]
1848 [D loss: 1.012331, acc.: 50.00%] [G loss: 0.364075]
1849 [D loss: 1.079406, acc.: 50.00%] [G loss: 0.389512]
1850 [D loss: 1.022636, acc.: 50.00%] [G loss: 0.402182]
1851 [D loss: 1.019700, acc.: 50.00%] [G loss: 0.420095]
1852 [D loss: 1.064587, acc.: 50.00%] [G loss: 0.263418]
1853 [D loss: 0.995768, acc.: 50.00%] [G loss: 0.318325]
1854 [D loss: 1.026917, acc.: 50.00%] [G loss: 0.370538]
1855 [D loss: 1.085267, acc.: 50.00%] [G loss: 0.387905]
1856 [D loss: 1.051751, acc.: 50.00%] [G loss: 0.399965]
1857 [D loss: 1.059315, acc.: 50.00%] [G loss: 0.307671]
1858 [D loss: 1.119134, acc.: 50.00%] [G loss: 0.359710]
1859 [D loss: 1.034531, acc.: 50.00%] [G loss: 0.401007]
1860 [D loss: 0.975769, acc.: 50.00%] [G loss: 0.430736]
1861 [D loss: 1.071377, acc.: 50.00%] [G loss: 0.343053]
1862 [D loss: 1.019130, acc.: 5

1989 [D loss: 1.100084, acc.: 50.00%] [G loss: 0.367248]
1990 [D loss: 1.074538, acc.: 50.00%] [G loss: 0.304685]
1991 [D loss: 1.111658, acc.: 50.00%] [G loss: 0.314876]
1992 [D loss: 1.129676, acc.: 50.00%] [G loss: 0.366546]
1993 [D loss: 0.974656, acc.: 50.00%] [G loss: 0.469976]
1994 [D loss: 1.025209, acc.: 50.00%] [G loss: 0.305826]
1995 [D loss: 0.996067, acc.: 50.00%] [G loss: 0.284171]
1996 [D loss: 1.176491, acc.: 50.00%] [G loss: 0.299733]
1997 [D loss: 1.037211, acc.: 50.00%] [G loss: 0.419427]
1998 [D loss: 1.051221, acc.: 50.00%] [G loss: 0.326973]
1999 [D loss: 1.044539, acc.: 50.00%] [G loss: 0.306188]
2000 [D loss: 1.021422, acc.: 50.00%] [G loss: 0.302407]


In [122]:
gan.generate_midi_from_saves(possibleNotes)

Composing from ../output/GAN_1570439027\GANmodel_weights_0.hdf5
[[[-0.94293034]
  [ 0.9511678 ]
  [ 0.95636314]
  [-0.21992657]
  [-0.6503713 ]
  [ 0.99673396]
  [ 0.9872717 ]
  [ 0.7317666 ]
  [-0.6498992 ]
  [ 0.23074849]
  [-0.1567165 ]
  [ 0.89335775]
  [-0.731017  ]
  [ 0.9066702 ]
  [ 0.15424827]
  [ 0.9639665 ]
  [ 0.9858628 ]
  [ 0.974552  ]
  [ 0.6970632 ]
  [-0.44110644]
  [ 0.20285414]
  [-0.9963693 ]
  [ 0.10762044]
  [ 0.64688444]
  [-0.69539547]
  [ 0.31526315]
  [-0.8685319 ]
  [ 0.9949453 ]
  [-0.96725065]
  [ 0.10382608]
  [ 0.5636406 ]
  [-0.94493407]
  [ 0.9792849 ]
  [ 0.59865683]
  [ 0.13042538]
  [ 0.96497583]
  [-0.98510236]
  [ 0.616735  ]
  [ 0.962668  ]
  [ 0.92874897]
  [ 0.9901613 ]
  [-0.52822757]
  [-0.47610646]
  [-0.9229445 ]
  [ 0.23479456]
  [-0.99814975]
  [-0.9073559 ]
  [-0.9771247 ]
  [-0.47023472]
  [ 0.8450416 ]
  [-0.29251167]
  [-0.72006226]
  [ 0.44590446]
  [-0.8185839 ]
  [-0.96681917]
  [ 0.5594001 ]
  [ 0.9965545 ]
  [ 0.8174353 ]
  [ 0.64

../output/GAN_1570439027/GAN_output_4
Composing from ../output/GAN_1570439027\GANmodel_weights_1300.hdf5
[[[ 0.9959104 ]
  [ 0.9725371 ]
  [ 0.4277731 ]
  [ 0.6466447 ]
  [-0.2397645 ]
  [-0.31211254]
  [-0.4867928 ]
  [-0.99711585]
  [-0.3768475 ]
  [ 0.8301846 ]
  [-0.3029596 ]
  [ 0.5598421 ]
  [ 0.5407779 ]
  [-0.01950943]
  [-0.8009113 ]
  [ 0.91839206]
  [-0.6776985 ]
  [-0.8608296 ]
  [-0.2922929 ]
  [ 0.9313903 ]
  [-0.8628891 ]
  [-0.9652906 ]
  [-0.98599225]
  [-0.8582042 ]
  [-0.03185019]
  [-0.8096114 ]
  [-0.6344647 ]
  [-0.31025332]
  [ 0.8177382 ]
  [ 0.32978398]
  [-0.8172393 ]
  [-0.8706268 ]
  [-0.45500994]
  [ 0.19768581]
  [ 0.9029304 ]
  [ 0.59354913]
  [ 0.2643258 ]
  [-0.8443395 ]
  [ 0.05900037]
  [-0.83505774]
  [ 0.6667676 ]
  [-0.7955165 ]
  [-0.14817132]
  [-0.7497144 ]
  [ 0.9561483 ]
  [ 0.93773234]
  [ 0.9599365 ]
  [ 0.8961353 ]
  [ 0.8280164 ]
  [ 0.36768773]
  [-0.89846784]
  [ 0.96327704]
  [-0.5269395 ]
  [ 0.9641858 ]
  [ 0.9415521 ]
  [ 0.7080726 ]

[[[ 0.995284  ]
  [ 0.54836345]
  [ 0.99779606]
  [ 0.816773  ]
  [ 0.12112167]
  [ 0.9028896 ]
  [-0.836984  ]
  [-0.49362987]
  [-0.9104736 ]
  [-0.9746636 ]
  [-0.32649818]
  [ 0.8401226 ]
  [-0.7671018 ]
  [-0.926077  ]
  [ 0.8219258 ]
  [ 0.5970247 ]
  [-0.7947639 ]
  [-0.5498568 ]
  [ 0.45789656]
  [ 0.8540513 ]
  [ 0.38306278]
  [ 0.42673478]
  [ 0.02450535]
  [ 0.8834447 ]
  [-0.34574944]
  [-0.82318693]
  [-0.18124941]
  [ 0.36039484]
  [-0.72288764]
  [ 0.80476207]
  [ 0.90168434]
  [-0.89069605]
  [-0.9989071 ]
  [ 0.5268345 ]
  [ 0.25753742]
  [-0.6400441 ]
  [-0.5927043 ]
  [ 0.9612405 ]
  [ 0.06259491]
  [-0.45570564]
  [-0.74284804]
  [-0.24707569]
  [ 0.32256004]
  [ 0.7661398 ]
  [-0.01301926]
  [ 0.8558969 ]
  [-0.7850193 ]
  [ 0.96592945]
  [-0.95488614]
  [-0.9710496 ]
  [ 0.8272846 ]
  [-0.9614878 ]
  [ 0.2662847 ]
  [-0.8359278 ]
  [ 0.8097777 ]
  [-0.62775123]
  [ 0.55750465]
  [ 0.03078178]
  [ 0.21106218]
  [ 0.97799844]
  [-0.13217326]
  [-0.7452792 ]
  [ 0.94

[[[ 0.97451854]
  [-0.69874775]
  [ 0.9469765 ]
  [-0.6900997 ]
  [-0.7647221 ]
  [ 0.04223719]
  [-0.889413  ]
  [-0.8161018 ]
  [ 0.36718702]
  [-0.34000787]
  [-0.20417714]
  [-0.5971832 ]
  [-0.1212225 ]
  [ 0.8490779 ]
  [-0.6295926 ]
  [ 0.39084482]
  [ 0.04012639]
  [-0.55831397]
  [ 0.35601702]
  [ 0.9843732 ]
  [ 0.5023697 ]
  [-0.2765126 ]
  [ 0.3541942 ]
  [ 0.4019861 ]
  [-0.5220711 ]
  [ 0.74961567]
  [ 0.9826397 ]
  [-0.48996878]
  [ 0.6116426 ]
  [-0.00519587]
  [ 0.662876  ]
  [-0.38731268]
  [-0.12436269]
  [ 0.13401459]
  [-0.5486989 ]
  [-0.2848232 ]
  [ 0.46502182]
  [-0.5435326 ]
  [ 0.18193091]
  [ 0.46246475]
  [-0.675353  ]
  [ 0.14148118]
  [-0.51159894]
  [ 0.98774344]
  [-0.27734083]
  [ 0.5016268 ]
  [-0.18938245]
  [-0.75955355]
  [ 0.8700379 ]
  [-0.41848293]
  [-0.4404454 ]
  [-0.19864926]
  [-0.95759344]
  [-0.0616031 ]
  [ 0.3033899 ]
  [ 0.51998234]
  [-0.90784645]
  [ 0.35259104]
  [-0.41759104]
  [ 0.9059249 ]
  [-0.4141998 ]
  [-0.7611893 ]
  [-0.94

../output/GAN_1570439027/GAN_output_16
Composing from ../output/GAN_1570439027\GANmodel_weights_600.hdf5
[[[-0.997128  ]
  [-0.99841547]
  [-0.9909693 ]
  [-0.9984451 ]
  [-0.98723966]
  [-0.87888235]
  [-0.86412007]
  [-0.42767024]
  [-0.8271749 ]
  [-0.67616916]
  [-0.9625199 ]
  [-0.2515185 ]
  [ 0.9886222 ]
  [-0.94185525]
  [ 0.574375  ]
  [-0.8664497 ]
  [ 0.7815784 ]
  [ 0.0318366 ]
  [ 0.4118354 ]
  [-0.5661387 ]
  [ 0.5390955 ]
  [-0.5672984 ]
  [ 0.99622166]
  [-0.99421936]
  [ 0.9579597 ]
  [ 0.7425308 ]
  [ 0.96932274]
  [-0.8456208 ]
  [ 0.9126273 ]
  [ 0.339567  ]
  [ 0.91360897]
  [ 0.99301374]
  [-0.05854364]
  [-0.9872319 ]
  [-0.9933573 ]
  [-0.9440137 ]
  [-0.6395689 ]
  [ 0.7651999 ]
  [-0.04600135]
  [ 0.08682454]
  [-0.55084574]
  [ 0.43291503]
  [ 0.65934145]
  [ 0.6971731 ]
  [ 0.13200581]
  [ 0.9866702 ]
  [ 0.49140742]
  [ 0.21456607]
  [ 0.66956294]
  [ 0.4305897 ]
  [-0.9701034 ]
  [ 0.07403346]
  [ 0.59496933]
  [-0.51457286]
  [ 0.8677479 ]
  [ 0.5297437 ]

../output/GAN_1570439027/GAN_output_20


In [111]:
count = 0
for model_path in glob.glob(gan.outputDest + "*.hdf5"):
    print("Composing from %s" % model_path)
    gan.generator.load_weights(model_path)
    prediction_notes = gan.generate_notes(possibleNotes)
    gan.create_midi(prediction_notes, gan.outputDest + 'GAN_output_' + str(count))
    print(gan.outputDest + 'GAN_output_' + str(count))
    count += 1

Composing from ../output/GAN_1570437586\GANmodel_weights_0.hdf5
[[[-0.79037344]
  [ 0.70460176]
  [-0.28947598]
  [ 0.8814572 ]
  [-0.6107855 ]
  [-0.38538986]
  [ 0.07783478]
  [ 0.41338134]
  [-0.8948215 ]
  [ 0.7541267 ]
  [ 0.42156288]
  [ 0.82168484]
  [-0.877071  ]
  [-0.3693905 ]
  [ 0.06191757]
  [-0.41833857]
  [ 0.6497538 ]
  [ 0.9913109 ]
  [ 0.9248787 ]
  [-0.99073195]
  [ 0.75921595]
  [-0.38791367]
  [ 0.7617763 ]
  [-0.08390729]
  [ 0.38214526]
  [-0.53922236]
  [ 0.99005866]
  [ 0.86944044]
  [ 0.02686271]
  [-0.9834003 ]
  [-0.44417801]
  [ 0.947354  ]
  [-0.4138857 ]
  [ 0.30625492]
  [ 0.8054296 ]
  [ 0.8645084 ]
  [-0.91852987]
  [-0.82418436]
  [-0.9886027 ]
  [ 0.2925736 ]
  [-0.9160985 ]
  [ 0.85405934]
  [ 0.70144534]
  [ 0.9964825 ]
  [ 0.19647111]
  [ 0.9931008 ]
  [-0.90167433]
  [ 0.8092278 ]
  [ 0.9887396 ]
  [ 0.9437407 ]
  [-0.97892994]
  [-0.27275044]
  [-0.42333934]
  [-0.60200846]
  [ 0.9684587 ]
  [-0.89766115]
  [ 0.79444194]
  [ 0.92340887]
  [-0.90

[[[-0.31863266]
  [-0.04901373]
  [ 0.9647185 ]
  [-0.9970886 ]
  [-0.7901506 ]
  [-0.5245192 ]
  [-0.99598765]
  [-0.98946106]
  [-0.97796994]
  [-0.7766503 ]
  [-0.9589379 ]
  [-0.90780485]
  [-0.89653635]
  [-0.83506596]
  [-0.9946219 ]
  [-0.698596  ]
  [ 0.70834357]
  [ 0.33375612]
  [-0.83358085]
  [-0.9176603 ]
  [-0.88028145]
  [-0.8599078 ]
  [ 0.9162866 ]
  [ 0.13114238]
  [-0.6726755 ]
  [-0.8211164 ]
  [-0.46654317]
  [ 0.22901817]
  [-0.96735495]
  [-0.28233814]
  [ 0.6000631 ]
  [ 0.6072935 ]
  [ 0.7140057 ]
  [ 0.82475835]
  [-0.14915876]
  [-0.96274424]
  [ 0.8794212 ]
  [-0.44175673]
  [ 0.39304227]
  [ 0.7064495 ]
  [-0.8201694 ]
  [ 0.12927328]
  [-0.85720026]
  [ 0.00342063]
  [ 0.10993685]
  [ 0.8549226 ]
  [-0.8549247 ]
  [-0.9479352 ]
  [-0.92161894]
  [-0.8428619 ]
  [ 0.01836698]
  [ 0.88301367]
  [ 0.4289415 ]
  [-0.65191066]
  [ 0.9797393 ]
  [ 0.300754  ]
  [-0.27318373]
  [-0.23616558]
  [ 0.92543495]
  [-0.91959906]
  [ 0.1948324 ]
  [ 0.7325499 ]
  [ 0.99

../output/GAN_1570437586/GAN_output_9
Composing from ../output/GAN_1570437586\GANmodel_weights_90.hdf5
[[[-0.99372005]
  [-0.07109336]
  [ 0.50417745]
  [ 0.6114273 ]
  [-0.99372274]
  [-0.93069106]
  [ 0.14586872]
  [-0.9409094 ]
  [-0.89703465]
  [-0.6764112 ]
  [ 0.91217065]
  [-0.08329976]
  [-0.6026658 ]
  [-0.9155475 ]
  [-0.4573485 ]
  [-0.2710771 ]
  [-0.83602333]
  [-0.9268878 ]
  [ 0.4406684 ]
  [ 0.5563792 ]
  [ 0.9190806 ]
  [-0.56058013]
  [ 0.567446  ]
  [ 0.8114215 ]
  [ 0.8143599 ]
  [-0.43429366]
  [-0.43725628]
  [ 0.1867322 ]
  [-0.52914596]
  [-0.9893395 ]
  [ 0.60228133]
  [-0.9499852 ]
  [-0.8755273 ]
  [ 0.9668063 ]
  [-0.04185651]
  [-0.33947974]
  [ 0.90217185]
  [ 0.85657346]
  [ 0.14158826]
  [-0.539097  ]
  [-0.33338523]
  [-0.90614957]
  [ 0.562485  ]
  [ 0.6537802 ]
  [ 0.3161599 ]
  [ 0.01375579]
  [-0.38288748]
  [ 0.56375265]
  [ 0.90417427]
  [ 0.99968565]
  [ 0.92991376]
  [ 0.87509125]
  [ 0.91637343]
  [ 0.95765615]
  [-0.5329507 ]
  [-0.8507352 ]
 

## Generating Music

I will now use the model to generate music by feeding it a random string of notes and have it predict the next one, then have it predict the one after that until a full song has been generated.

In [7]:
def generate_notes(model, notes, network_input, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    pitchnames = sorted(notes)
    
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)
#         prediction_input = (prediction_input / float(n_vocab))*2 - 1

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        
        pattern = np.append(pattern,index)
        pattern = pattern[1:len(pattern)]
    
    print(prediction_output)

    return prediction_output

# prediction_output = generate_notes(model, possibleNotes, network_input, n_vocab)

Next, I will create a midi using these notes and save to a file

In [8]:
def create_midi(prediction_output, filename):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='{}.mid'.format(filename))
    
# create_midi(prediction_output, outputDest + 'LSTM_output_final')

Alternatively, I can run this script to convert all of the models into midi files and select my favourite from a much larger album.

In [ ]:
# Have each model make a song
count = 0
for model_path in glob.glob(outputDest + "*.hdf5"):
    print("Composing from %s" % model_path)
    model.load_weights(model_path)
    prediction_output = generate_notes(model, possibleNotes, network_input, n_vocab)
    create_midi(prediction_output, outputDest + 'LSTM_output_' + str(count))
    print(outputDest + 'LSTM_output_' + str(count))
    count += 1